In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import bqplot as bq
import matplotlib as mpl
import ipywidgets
%matplotlib inline

We are considering data on regional real estate listing prices from Zillow. The data was
acquired from [Zillow.com/data](https://zillow.com/data) on (2019-03-30). Zillow holds the copyrights on the aggregated data and makes this aggregated data available freely for non-commercial use. Of the data, we are utilizing state and city level time series of single family residence listing prices. The data contains the median listing price collected every month across states and cities. The data comprises of several csv files and a data dictionary. We utilize the state level data which has around 13000 rows and 82 columns. The city level data comprises of 3.7 million rows and 85 columns. To avoid memory issues, we subset this analysis to Illinois only.

We begin by considering which month of the year is suitable for listing/buying a house. We consider the median prices of houses per month of year and try to see if there are any trends.  We are doing this on a state by state basis as a first pass.

In [2]:
states = pd.read_csv("State_time_series.csv")

In [3]:
states.head()

,Date,RegionName,DaysOnZillow_AllHomes,InventorySeasonallyAdjusted_AllHomes,InventoryRaw_AllHomes,MedianListingPricePerSqft_1Bedroom,MedianListingPricePerSqft_2Bedroom,MedianListingPricePerSqft_3Bedroom,MedianListingPricePerSqft_4Bedroom,MedianListingPricePerSqft_5BedroomOrMore,...,ZHVI_BottomTier,ZHVI_CondoCoop,ZHVI_MiddleTier,ZHVI_SingleFamilyResidence,ZHVI_TopTier,ZRI_AllHomes,ZRI_AllHomesPlusMultifamily,ZriPerSqft_AllHomes,Zri_MultiFamilyResidenceRental,Zri_SingleFamilyResidenceRental
0,1996-04-30,Alabama,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,45600.0,99500.0,79500.0,79000.0,140200.0,NaN,NaN,NaN,NaN,NaN
1,1996-04-30,Arizona,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,67100.0,78900.0,103600.0,107500.0,168700.0,NaN,NaN,NaN,NaN,NaN
2,1996-04-30,Arkansas,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,38400.0,70300.0,64400.0,64500.0,115200.0,NaN,NaN,NaN,NaN,NaN
3,1996-04-30,California,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,95100.0,136100.0,157900.0,162000.0,270600.0,NaN,NaN,NaN,NaN,NaN
4,1996-04-30,Colorado,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,82700.0,99400.0,128100.0,133600.0,209300.0,NaN,NaN,NaN,NaN,NaN


Converting the date to a datetime for ease of analysis

In [4]:
states.Date = pd.to_datetime(states.Date)

In [5]:
sc_x1 = bq.OrdinalScale()
sc_y1 = bq.LinearScale()
ax_x1 = bq.Axis(label='Month of year', scale=sc_x1)
ax_y1 = bq.Axis(label='Median price', scale=sc_y1, orientation='vertical')

month_names = ['Jan', 'Feb', 'Mar', 'Apr', 'May', 'Jun',
               'Jul', 'Aug', 'Sep', 'Oct', 'Nov', 'Dec']
y = []
for month in range(1, 13):
    y.append(list(states
                  .MedianListingPrice_SingleFamilyResidence
                  .groupby(states.Date.dt.month)
                  .get_group(month)
                  .dropna()))
    
boxes = bq.Boxplot(x=month_names, y=y,
                   scales={'x': sc_x1, 'y': sc_y1},
                   box_fill_color='orange',
                   stroke ='green',
                   outlier_fill_color='blue')

fig1 = bq.Figure(axes=[ax_x1, ax_y1], marks=[boxes])

In [6]:
state_select1 = ipywidgets.Dropdown(options= ['All'] + list(states.RegionName.unique()),
                                    value='All',
                                    description='State')

In [7]:
def on_change(change):
    acc = []
    state = state_select1.value
    filter_ = pd.Series(True, index=states.index) # select all rows
    if state != 'All':
        filter_ = filter_ & (states['RegionName']==state)   
    df = states.loc[filter_]
    grpd = df.MedianListingPrice_SingleFamilyResidence.groupby(df.Date.dt.month)

    for month in range(1, 13):
        acc.append(list(grpd.get_group(month).dropna().values))
    boxes.y = acc
          
        
state_select1.observe(on_change, 'value')


In [8]:
ipywidgets.VBox([ipywidgets.HBox([state_select1]), fig1])

OK, so the answer to our question of best month for selling seems to vary by state. Overall May seems to be a good bet for sellers and January for buyers.

Diving deeper into city level, we can see how the timeseries for home prices varies across time. For memory reasons, we filter thecities to Illinois only.

In [9]:
# cols = ['Date', 'RegionName', 'MedianListingPrice_SingleFamilyResidence']
# city = pd.read_csv('City_time_series.csv', usecols = cols)

In [10]:
# city.memory_usage()

In [11]:
# crosswalk = pd.read_csv('cities_crosswalk.csv')

In [12]:
# city = city.merge(crosswalk, left_on=['RegionName'], right_on=['Unique_City_ID']).drop(columns=['Unique_City_ID'])

In [13]:
# city.to_csv('city_sfrlist_ts.csv', index = False)

In [14]:
city = pd.read_csv('city_sfrlist_ts.csv')

Filter for IL only

In [15]:
city = city.loc[city.State=='IL'].copy()

In [16]:
city.Date = pd.to_datetime(city.Date)

In [17]:
grpd = city.groupby(['RegionName', city.Date.dt.year])

city_median_lp_year= grpd.agg({
    'MedianListingPrice_SingleFamilyResidence':pd.Series.median,
    'City':lambda x: x.iloc[0],
    'County': lambda x: x.iloc[0],
    'State': lambda x: x.iloc[0]
         })

In [18]:
city_median_lp_year.reset_index(inplace=True)

In [19]:
sc_x2 = bq.OrdinalScale()
sc_y2 = bq.LinearScale()
ax_x2 = bq.Axis(label='Year', scale=sc_x2, tick_rotate=90)
ax_y2 = bq.Axis(label='Median price', scale=sc_y2,
                orientation='vertical', label_offset='3.5em')


lines = bq.Lines(x=city_median_lp_year.loc[city_median_lp_year.City=='Champaign'].Date,
                 y = city_median_lp_year.loc[city_median_lp_year.City=='Champaign'].MedianListingPrice_SingleFamilyResidence,
                 scales={'x': sc_x2, 'y': sc_y2}, 
                 markers = 'circle',
                 )

fig2 = bq.Figure(axes=[ax_x2, ax_y2], marks=[lines])

In [20]:
# h = city_median_lp_year.head()

In [21]:
cities = (city_median_lp_year.dropna(subset=['MedianListingPrice_SingleFamilyResidence'], axis=0)
          .groupby('RegionName')
          .City
          .first()
          .values)

In [22]:
city_dropdown = ipywidgets.Dropdown(options=[_ + ', IL' for _ in cities],
                                    value='Champaign, IL',
                                    description='City' )

In [23]:
def on_change(change):
    city_state = city_dropdown.value #Urbana, IL
    city, state = city_state.split(', ')
    filter_ = ((city_median_lp_year.City==city)& (city_median_lp_year.State == state)
                &(city_median_lp_year.MedianListingPrice_SingleFamilyResidence.notnull()))
    x = city_median_lp_year.loc[filter_,'Date']
    y = city_median_lp_year.loc[filter_, 'MedianListingPrice_SingleFamilyResidence']
    lines.x = x
    lines.y = y

city_dropdown.observe(on_change, 'value')


In [24]:
ipywidgets.VBox([city_dropdown, fig2])

This visualization shows that there is a dip in the home prices around 2012-2014. It would be interesting to correlation with the subprime mortagage crisis. 

For the third visualization, we explore how the median listing price varies across states across years. To do this on a map, we need to use bqplot Map marker. The Map marker can be colored by setting values for the state FIPS code and passing a dictionary.

In [26]:
# Get the FIPS Code and remove spaces from the state names to match what is in our dataset
fips_codes = pd.read_csv('https://raw.githubusercontent.com/kjhealy/fips-codes/master/state_fips_master.csv')
fips_codes.state_name = fips_codes.state_name.str.replace(' ','')
fips_codes.set_index("state_name", inplace=True)


In [27]:
# group timeseries by state and year
states_grouped = (states
                  .groupby(['RegionName', states.Date.dt.year])
                  .MedianListingPrice_SingleFamilyResidence
                  .median()
                  .dropna())

# helper function for getting a dict to pass to bqplot
def get_state_values(year=2017):
    d = states_grouped.xs(key=year, level='Date').to_dict()
    # convert state names in keys to FIPS codes
    return {fips_codes.loc[state, 'fips']:value for state, value in d.items() if state in fips_codes.index}



In [36]:
cscale = bq.ColorScale(scheme='plasma',
                       min=100000,
                       max=500000,
                       reverse=True# reverse the colorscale or Hawaii is not visible
                      )
map_tt = bq.Tooltip(labels = ['State', 'Median List Price'], fields = ['name', 'color'])
sc_geo = bq.AlbersUSA()
states_map = bq.Map(color=get_state_values(year=2017),
                    colors={'default_color':'black'},
                    map_data=bq.topo_load('map_data/USStatesMap.json'),
                    scales={'projection': sc_geo, 'color':cscale}, tooltip = map_tt)
cax = bq.ColorAxis(scale=cscale, orientation='vertical', side='left', label='Listing Price')
fig3 = bq.Figure(marks=[states_map], title='Median single family residence listing prices'
                 , axes=[cax])


In [37]:
#year_slider = ipywidgets.IntSlider(min=1996, max=2017, value=2017)
# we do not have much data before 2010. 
year_slider = ipywidgets.IntSlider(min=2010, max=2017, value=2017)

In [38]:
def on_change_year(value):
    year = year_slider.value
    try:
        states_map.color = get_state_values(year=year)
    except Exception:
        return

year_slider.observe(on_change_year, 'value')

In [39]:
ipywidgets.VBox([year_slider, fig3])

The dataset from Zillow allows us to see the evolution of the trajectories of the housing market across the United States. The data can be used to investigate a variety of questions such as when it makes sense to rent vs buy, the impact of the subprime mortgage crisis and how real estate fared as an investment. From the data, it is clear that the midwest typically has lower housing costs compared to the east and the west coasts. Also, each state has a different seasonal cycle of housing costs with Illinois having a predominant peak during the summer and states such as Delaware, Colorado having near uniform demand throughout the year. Several cites in Illinois experienced a dip in values during 2012-2014 which seems to coincide with the subprime mortagage crisis. A more thorough analysis would need to dive deeper into the county level and also see how rents have fared during the same period. 